In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sys
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [51]:
# Funciones

# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
def EncoderViewMinable(df):
    new_dataset = df
    norm = MinMaxScaler()
    norm = norm.fit(new_dataset.values[:,:])
    valMin = norm.data_min_
    valMax = norm.data_max_
    dataRange = norm.data_range_
    #df_norm = norm.fit_transform(df.values[:,:-1])

    return [valMin, valMax, dataRange]


'''
Funcion para generar el vector de pesos aleatorio.
Entradas:
w: Vector de pesos w [0,1], igual al numero de caracteristicas normalizadas.
nc: Nunero de ceros que debe contener el vector de pesos [10,20,30,40,50]
'''


def GenerateWeightVector(w, nc):
    posceros = np.random.choice(len(w), nc, replace=False)
    w[posceros] = 0
    s  = np.sum(w)
    wf = np.round(w/s, 4)
    return wf






'''
-Función de calidad, que retorna la metrica de calidad asociada a ese vector w especifico
-Se debe tener en cunata la seleccion de la metrica de calidad asociada, para evaluar
el desempeño del algoritmo (Problema de Clasificacion)
Entradas:
    df_norm: Dataset Normalizado.
    wi: Vector de Pesos.
'''

'''
# Dependiendo del vector de pesos me extrae el acuracy - F1 score
def qualityFunction(df_norm, wi,df):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    for i in range(len(df_norm)):
        minDep = sys.float_info.max
        posMinDep = 0
        vrf = df_norm[i]
        print(f"Vector {i} :", vrf) 
        for j in range(len(df_norm)):
            if i != j:
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                print("Distancia Euclideana: ", dE)
                if dE < minDep:
                    posMinDep=j
                    minDep = dE
        print(f"vector {i} es muy similar el vector en la posición {posMinDep}, con distancia de {minDep}")
        y_pred.append(df.values[posMinDep][-1])
    qs = accuracy_score(df.values[:,-1], y_pred)
    #mc = confusion_matrix(df.values[:,-1], y_pred)
    #print("Matriz de Confusión: ", mc)

    return [qs, y_pred]
'''



'''
Función para generar la memoria Armonica
Entradas:
    MAC: Tamaño de la  Memoria Armonica
    wi: Vector de Pesos.
    nc: Numero de ceros (Selección de atributos)
'''

def GenerateArmonyMemory(df_norm, MAC,nc,df):
    Lw = []
    #np.random.seed(123)
    for i in range (MAC):
        # Creo la semilla
        
        vp = np.random.rand(df_norm.shape[1])
        #print("Vector de pesos Original: ", vp)
        wi = GenerateWeightVector(vp, nc)
        #print("Vector de pesos Normalizado: ", wi)
        Qs = qualityFunction(df_norm, wi,df)
        #print("Qs: ",Qs )
        wiq = np.append(wi, Qs)
        Lw.append(wiq)

    
    Lw.sort(key=lambda x: x[-1], reverse=True)
    return Lw



def ImprovisationGBHS(PAR, hmn, nc,dataset_normalizado , lmp, HMRC,df):

    df_norm = dataset_normalizado
    np.random.seed(123)
    # Se genera la memoria Armonica - diferentes tamaños
    MA = GenerateArmonyMemory(df_norm,hmn,nc,df)
    #print("MA: ", MA)
    P=len(MA[0]) 

    curvaFitnes = []
    vectorIteration= []
    for i in range (lmp):
        pesosAleatorios = np.random.rand(P-1)
        for j in range(P-1):
            Aleatorio1 = random.random()
            #print("Aleatorio 1: ", Aleatorio1) 
            if (Aleatorio1 < HMRC):
                pma = random.randint(0, hmn-1)
                pesosAleatorios[j]= MA[pma][j]

                Aleatorio2 = random.random()
                if Aleatorio2 < PAR:
                    pesosAleatorios[j] = MA[0][j]
            
            else:
                Aleatorio3 = random.random()
                if Aleatorio3 < nc/P:
                    Aleatorio4 = 0
                else:
                    Aleatorio4 = Aleatorio3/(P-nc)
                
                pesosAleatorios[j] = Aleatorio4
        

        # Normalización de los pesos
        wf = GenerateWeightVector(pesosAleatorios, 0)
        #print("Nuevo vector analizar el fitnes: ", wf)
        fitnes = qualityFunction(df_norm, wf,df)
        #print("Nuevo Fitnes: ", fitnes)



        # Remplazo
        if MA[hmn -1][P-1] < fitnes:
            new_register = np.append(wf, fitnes)
            MA[hmn-1] = new_register
            #print("------------------------------------")
            MA.sort(key=lambda x: x[-1], reverse=True)
        

        
        curvaFitnes.append(MA[0][P-1])
        vectorIteration.append(MA[0])
        

    
    '''
    print("Valor de la curva en la ultima poisción: ",curvaFitnes[-1])

    dicc = {"vector":vectorIteration,
                "Fitnes": curvaFitnes}
    

    df_new = pd.DataFrame(data=dicc)
    df_new.to_csv(f"ResultadosImprovisacion/GBHS.csv")
    dicc = dict()
    '''
    return [curvaFitnes, vectorIteration[-1]]


# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
# df: es la matriz df.values [] , no incluye la etiqueta del grupo
def NormalizeViewMinable(df,valMin, dataRange):
    dataset_normalizado = np.empty((df.shape[0], df.shape[1]))
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            dataset_normalizado[i][j]= (df[i][j] - valMin[j])/dataRange[j]

    return dataset_normalizado




In [1]:
# Pruebs
d = {'A': [5.1,0.9,4.7,7,6.4,6.9,5.5,6.3,5.8,7.1],
        'B': [3.5,3,3.2,3.2,3.2,3.1,2.3,3.3,2.7,3],
        'C': [1.4,1.4,1.3,4.7,4.5,4.9,4,6,5.1,5.9],
        'D': [0.2,0.2,0.2,1.4,1.5,1.5,1.3,2,1.9,2.1],
        'G': [0,0,0,1,1,1,1,2,2,2]}

df = pd.DataFrame(data=d)
print(df)
# 1. Codificacion 
valMin, valMax, dataRange = EncoderViewMinable(df)
print("Valores minimos: ", valMin)
print("Rangos de Valores: ", dataRange)
print("Longitudes : ", len(valMin), len(valMax), len(dataRange))
#2. Normalizamos los datos
df_normalizado = NormalizeViewMinable(df.values[:, :-1],valMin, dataRange)
print(df_normalizado)



NameError: name 'pd' is not defined

In [36]:
# 3 Generar vector de pesos
np.random.seed(0)
vp = np.random.rand(4)
print("Vector de pesos Original: ", vp)
wi = GenerateWeightVector(vp, 0)
print(wi)

Vector de pesos Original:  [0.5488135  0.71518937 0.60276338 0.54488318]
[0.2276 0.2966 0.2499 0.2259]


In [37]:
def qualityFunction(df_norm, wi,df,k):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    for i in range(len(df_norm)):
        vrf = df_norm[i]
        print(f"Vector {i} :", vrf) 
        ListaPesosPonderados= [[sys.float_info.max,0] for a in range(k)]
        for j in range(len(df_norm)):
            if i != j:
                print(ListaPesosPonderados)
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                print("Distancia Euclideana: ", dE)
                if dE < ListaPesosPonderados[k-1][0]:
                    ListaPesosPonderados[k-1][0]=dE
                    ListaPesosPonderados[k-1][1]=j
                    ListaPesosPonderados.sort(key=lambda x: x[0], reverse=False)
        print(f"vector {i} es muy similar a los vectores en la posición {ListaPesosPonderados}")

        grupos = []
        for i in range(len(ListaPesosPonderados)):
            index=ListaPesosPonderados[i][1]
            g = df.values[index][-1]
            grupos.append(g)
        
        print("Grupos asociados: ", grupos)
        grupoSelected = int(pd.Series(grupos).value_counts().index[0])
        print("Grupo seleccionado: ", grupoSelected)
        y_pred.append(grupoSelected)
    qs = accuracy_score(df.values[:,-1], y_pred)
    #mc = confusion_matrix(df.values[:,-1], y_pred)
    #print("Matriz de Confusión: ", mc)

    return [qs, y_pred]



def qualityFunctionOriginal(df_norm, wi,df):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    minDep = sys.float_info.max
    for i in range(len(df_norm)):
        posMinDep = 0
        minDep = sys.float_info.max 
        vrf = df_norm[i]
        for j in range(len(df_norm)):
            if i != j:
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                if dE < minDep:
                    posMinDep=j
                    minDep = dE
        #print(posMinDep)
        y_pred.append(df.values[posMinDep][-1])
    qs = accuracy_score(df.values[:,-1], y_pred)
    return qs

In [49]:
# Funcion de calidad
calidad = qualityFunction(df_normalizado, wi,df,7)


Vector 0 : [0.67741935 1.         0.0212766  0.        ]
[[1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0]]
Distancia Euclideana:  0.15593800872933283
[[0.15593800872933283, 1], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0]]
Distancia Euclideana:  0.019597974626315823
[[0.019597974626315823, 2], [0.15593800872933283, 1], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0]]
Distancia Euclideana:  0.2532182152739098
[[0.019597974626315823, 2], [0.15593800872933283, 1], [0.2532182152739098, 3], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+308, 0], [1.7976931348623157e+30

In [50]:
calidad

[0.3, [1, 1, 1, 1, 1, 1, 2, 1, 1, 1]]

In [12]:
import pandas as pd
gs= [0.0, 0.0, 1.0]
c = pd.Series(gs).value_counts()
c.index[0]

0.0

In [14]:
a = pd.Series(gs).value_counts().index[0]
print(a)

0.0


In [80]:
import numpy as np

List_numbers = [2, 4, 7, 8, 12, 4, 6, 7, 8, 91, 8]
print(np.unique(List_numbers, return_counts=True))

(array([ 2,  4,  6,  7,  8, 12, 91]), array([1, 2, 1, 2, 3, 1, 1], dtype=int64))


In [56]:
# Prueba muestra dataset original
dff = pd.read_csv('DataOpt/Final_join_prueba.csv')
dff.head(5)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
0,4,47,89,60000,4,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
1,5,47,72,71000,55,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,4,46,78,60000,6,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
3,5,48,85,6000,4,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
4,6,46,77,80000,12,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0


In [58]:
# 1. Codificacion 
valMin, valMax, dataRange = EncoderViewMinable(dff)
print("Valores minimos: ", valMin)
print("Rangos de Valores: ", dataRange)
print("Longitudes : ", len(valMin), len(valMax), len(dataRange))
#2. Normalizamos los datos
df_normalizado = NormalizeViewMinable(dff.values[:, :-1],valMin, dataRange)
print(df_normalizado)

Valores minimos:  [ 3.000000e+00  4.200000e+01  6.900000e+01  6.000000e+03  4.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  2.300000e+01  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  1.000000e+00  9.000000e-01  2.000000e+00  5.000000e+00 -1.000000e+00
 -1.000000e+00 -1.000000e+00 -1.000000e+00  1.600000e+01  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
  0.000000e+00  0.000000e+00  3.180000e+01  2.336000e+01  2.819000e+01
  7.070000e+00  1.351205e+04  0.000000e+00  6.110000e+01  5.000000e-02
  7.724000e+01  3.257000e+01  2.321000e+01  2.789000e+01  7

C:\Users\germanm\AppData\Local\Temp\ipykernel_19400\3589246154.py:180: RuntimeWarning: invalid value encountered in double_scalars
  dataset_normalizado[i][j]= (df[i][j] - valMin[j])/dataRange[j]


In [67]:
dataRange

array([4.00000e+00, 7.00000e+00, 3.30000e+01, 7.40000e+04, 6.80000e+01,
       2.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       2.00000e+00, 2.00000e+00, 2.00000e+00, 0.00000e+00, 1.00000e+00,
       1.00000e+00, 5.00000e+00, 0.00000e+00, 1.20000e+01, 3.45000e+01,
       1.38000e+02, 2.40000e+01, 2.30000e+01, 3.75000e+01, 1.20000e+01,
       7.50000e+01, 9.75000e+01, 0.00000e+00, 1.00000e+00, 2.00000e+00,
       4.00000e+00, 5.10000e+00, 2.00000e+00, 2.00000e+00, 0.00000e+00,
       0.00000e+00, 2.80000e+01, 4.10000e+01, 5.20000e+01, 0.00000e+00,
       1.00000e+02, 0.00000e+00, 1.00000e+02, 6.33300e+01, 2.00000e+01,
       1.00000e+02, 1.00000e+02, 7.16700e+01, 0.00000e+00, 1.00000e+02,
       1.00000e+02, 0.00000e+00, 1.00000e+02, 1.00000e+02, 3.57100e+01,
       1.00000e+02, 0.00000e+00, 3.14000e+00, 1.71000e+00, 1.74000e+00,
       4.41000e+00, 2.98912e+03, 6.50000e-01, 3.53700e+02, 2.30000e-01,
       1.18100e+01, 2.57000e+00, 1.98000e+00, 1.83000e+00, 3.900

In [61]:
# 3 Generar vector de pesos longitud 174
np.random.seed(0)
vp = np.random.rand(174)
print("Vector de pesos Original: ", vp)
wi = GenerateWeightVector(vp, 0)
print(wi)

Vector de pesos Original:  [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203  0.3595079  0.43703195
 0.6976312  0.06022547 0.66676672 0.67063787 0.21038256 0.1289263
 0.31542835 0.36371077 0.57019677 0.43860151 0.98837384 0.10204481
 0.20887676 0.16130952 0.65310833 0.2532916  0.46631077 0.24442559
 0.15896958 0.11037514 0.65632959 0.13818295 0.19658236 0.36872517
 0.82099323 0.09710128 0.83794491 0.09609841 0.97645947 0.4686512
 0.97676109 0.60484552 0.73926358 0.03918779 0.28280696 0.12019656
 0.2961402  0.11872772 0.31798318 0.41426299 0.0641475  0.69247212
 0.56660145 0.26538949 0.52324805 0.09

In [64]:
# Funcion de calidad
calidad = qualityFunction(df_normalizado, wi,dff,1)


Vector 0 : [0.25       0.71428571 0.60606061 0.72972973 0.         0.
        nan        nan        nan        nan 0.         0.5
 0.5               nan 0.         0.         0.4               nan
 0.         0.         0.5        0.         0.         0.
 0.         0.         0.                nan 0.         0.
 0.25       0.21568627 0.         0.                nan        nan
 0.75       0.46341463 0.07692308        nan 0.3               nan
 0.         0.         0.         0.         0.7        0.
        nan 1.         0.                nan 0.         0.
 0.         0.                nan 0.56050955 0.70175439 0.5
 0.43764172 0.60303367 0.58461538 0.2374894  0.2173913  0.22438611
 0.63035019 0.63636364 0.78688525 0.45128205 0.38677767 0.66197183
 0.57286432 0.77777778 0.19623234 0.45251397 0.80630631 0.80813953
 0.28884026 0.44781419 0.51282051 0.48976744 0.39130435 0.89634146
 1.         0.                nan 0.                nan        nan
        nan        nan 0.         0.  

In [66]:
calidad

[0.30434782608695654,
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [4]:
#  CARGAR EL DATASEET PRINCIPAL CON LOS  RESPECTIVOS GRUPOS
# ==============================================================================
df = pd.read_csv('Iris.csv')
df_groups = df.copy()
df1 = df.copy()
df1 = df1.drop(["species"], axis= 1)


In [5]:
df_groups['Grupo'] = df_groups['species'].map({'setosa':0, 'versicolor':1, 'virginica':2})
df_groups = df_groups.drop(["species"], axis= 1)
#dataset_train, dataset_test = train_test_split(df_groups, test_size = 0.05, random_state=43)
#print("Longitud Dataset Entrenamiento:",  dataset_train.shape)

In [6]:
valMin, valMax, dataRange = EncoderViewMinable(df1)
print("Longitudes : ", len(valMin), len(valMax), len(dataRange))
# Guardamos los encoders apra posteriores usos
np.savetxt('Encoder_ValMin.txt', valMin)
np.savetxt('Encoder_dataRange.txt', dataRange)



Longitudes :  4 4 4


In [10]:
# Definición de Pesos
# =================================================================================================

#1.Union grupos | unico dataset
#final_datset_join = dataset_train.reset_index(drop=True)
final_dataset_join= df_groups
#2.Normalizacion dataset

#df_norm = NormalizeViewMinable(final_datset_join.values[:,:-1],valMin, dataRange)
#print(df_norm.shape)


In [11]:
final_dataset_join

,sepal_length,sepal_width,petal_length,petal_width,Grupo
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [17]:
#print("Valmin: ", valMin)
#print("DataRange: ", dataRange)

In [18]:
#df_norm[0:5]

In [12]:
lmp = 10                                                                                
HMRC = 0.85                                          
PAR_array=[0.25,0.3, 0.35,0.40]                                                         
hmns_array = [5,10,15,20]  
nc_array = [0,1,2]                                        


#1. Se contruyen los grupos de calidad
#df =final_datset_join
# Lectura de los Encoders

df_norm = NormalizeViewMinable(final_dataset_join.values[:,:-1],valMin, dataRange)


cont = 1

# 
# GBHS - IMPROVISACIÓN
# ============================================================================================
for incero in range(len(nc_array)):
    for ihmn in range(len(hmns_array)):
        for ipar in range(len(PAR_array)):
            print(f"PAR: {PAR_array[ipar]}, Longitud de la Memoria Armonica: {hmns_array[ihmn]}, Numero de ceros: {nc_array[incero]} ")
            print(f"Iteracion {cont}")
            # Parametros Bucle
            PAR = PAR_array[ipar]
            hmn = hmns_array[ihmn]
            nc= nc_array[incero]

            # Se genera la memoria Armonica - diferentes tamaños
            #np.random.seed(43)
            MA = GenerateArmonyMemory(df_norm,hmn,nc,final_dataset_join)
            P=len(MA[0]) 

            curvaFitnes = []
            vectorIteration= []
            for i in range (lmp):
                pesosAleatorios = np.random.rand(P-1)
                for j in range(P-1):
                    Aleatorio1 = random.random() 
                    if (Aleatorio1 < HMRC):
                        pma = random.randint(0, hmn-1)
                        pesosAleatorios[j]= MA[pma][j]

                        Aleatorio2 = random.random()
                        if Aleatorio2 < PAR:
                            pesosAleatorios[j] = MA[0][j]
                    
                    else:
                        Aleatorio3 = random.random()
                        if Aleatorio3 < nc/P:
                            Aleatorio4 = 0
                        else:
                            Aleatorio4 = Aleatorio3/(P-nc)
                        
                        pesosAleatorios[j] = Aleatorio4
                

                # Normalización de los pesos
                wf = GenerateWeightVector(pesosAleatorios, 0)
                fitnes = qualityFunction(df_norm, wf, final_dataset_join)



                # Remplazo
                if MA[hmn -1][P-1] < fitnes:
                    new_register = np.append(wf, fitnes)
                    MA[hmn-1] = new_register
                    #print("------------------------------------")
                    MA.sort(key=lambda x: x[-1], reverse=True)
                

                
                curvaFitnes.append(MA[0][P-1])
                vectorIteration.append(MA[0])
                
 
            
            print("Valor de la curva en la ultima poisción: ",curvaFitnes[-1])

            dicc = {"vector":vectorIteration,
                        "Fitnes": curvaFitnes}
            
    
            df_new = pd.DataFrame(data=dicc)
            df_new.to_csv(f"Resultados/accuracy_PAR_{PAR}_Tmem_{hmn}_nc_{nc}.csv")
            cont=cont+1
            dicc = dict()

PAR: 0.25, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 1
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 2
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 3
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 4
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 10, Numero de ceros: 0 
Iteracion 5
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 10, Numero de ceros: 0 
Iteracion 6
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 10, Numero de ceros: 0 
Iteracion 7
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud d

C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 5, Numero de ceros: 2 
Iteracion 35
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 5, Numero de ceros: 2 
Iteracion 36
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 37
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 38


C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 39


C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 40
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 41
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 42
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 43
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 44
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 45


C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 46
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 47
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 48
Valor de la curva en la ultima poisción:  0.3333333333333333


In [55]:
#3.Optimización GBHS
# Se jecuta la mejor metahurtistica de acuerdo a los resultados obtenidos
metric,vector_pesos =  ImprovisationGBHS(0.35, 5, 2, df_norm, 50, 0.85, final_datset_join)
#metric,vector_pesos =  ImprovisationGBHS(0.4, 20, 50, df_norm, 30, 0.85, final_datset_join)
vector_pesos_optmizacion = vector_pesos[0: -1]
print("-------------------------- RESULTADOS OPTMIZACIÓN-------------------------")
print("Mejor Fitnes: ",metric)
print("Longitud del vector de pesos: ", len(vector_pesos_optmizacion))

Qs:  0.3380281690140845
Qs:  0.34507042253521125
Qs:  0.3380281690140845
Qs:  0.34507042253521125
Qs:  0.34507042253521125
-------------------------- RESULTADOS OPTMIZACIÓN-------------------------
Mejor Fitnes:  [0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.3450704

In [40]:
df_norm.shape

(142, 4)

In [38]:
MA = GenerateArmonyMemory(df_norm,5,1,final_datset_join)

vector 0 es muy similar el vector en la posición 34, con distancia de 0.0009604948689712412
vector 1 es muy similar el vector en la posición 24, con distancia de 0.00046701388888888964
vector 2 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 3 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 4 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 5 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 6 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 7 es muy similar el vector en la posición 119, con distancia de 0.0
vector 8 es muy similar el vector en la posición 119, con distancia de 0.0
vector 9 es muy similar el vector en la posición 119, con distancia de 0.0
vector 10 es muy similar el vector en la posición 119, con distancia de 0.0
vector 11 es muy similar el vector en la posición 119,

vector 43 es muy similar el vector en la posición 13, con distancia de 0.0
vector 44 es muy similar el vector en la posición 13, con distancia de 0.0
vector 45 es muy similar el vector en la posición 13, con distancia de 0.0
vector 46 es muy similar el vector en la posición 13, con distancia de 0.0
vector 47 es muy similar el vector en la posición 13, con distancia de 0.0
vector 48 es muy similar el vector en la posición 13, con distancia de 0.0
vector 49 es muy similar el vector en la posición 13, con distancia de 0.0
vector 50 es muy similar el vector en la posición 13, con distancia de 0.0
vector 51 es muy similar el vector en la posición 13, con distancia de 0.0
vector 52 es muy similar el vector en la posición 13, con distancia de 0.0
vector 53 es muy similar el vector en la posición 13, con distancia de 0.0
vector 54 es muy similar el vector en la posición 13, con distancia de 0.0
vector 55 es muy similar el vector en la posición 13, con distancia de 0.0
vector 56 es muy similar 

In [27]:
MA

[array([0.457     , 0.319     , 0.224     , 0.        , 0.34507042]),
 array([0.279     , 0.        , 0.253     , 0.47      , 0.34507042]),
 array([0.474     , 0.        , 0.198     , 0.328     , 0.34507042]),
 array([0.319     , 0.501     , 0.        , 0.18      , 0.34507042]),
 array([0.        , 0.269     , 0.213     , 0.518     , 0.33802817])]

In [33]:
df_norm

array([[0.33333333, 0.16666667, 0.45762712, 0.375     ],
       [0.16666667, 0.16666667, 0.38983051, 0.375     ],
       [0.02777778, 0.41666667, 0.05084746, 0.04166667],
       [0.69444444, 0.33333333, 0.6440678 , 0.54166667],
       [0.94444444, 0.33333333, 0.96610169, 0.79166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.47222222, 0.375     , 0.59322034, 0.58333333],
       [0.52777778, 0.375     , 0.55932203, 0.5       ],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.41666667, 0.33333333, 0.69491525, 0.95833333],
       [0.55555556, 0.54166667, 0.84745763, 1.        ],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.13888889, 0.41666667, 0.06779661, 0.08333333],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.5       , 0.25      , 0.77966102, 0.54166667],
       [0.52777778, 0.33333333, 0.6440678 , 0.70833333],
       [0.94444444, 0.25      ,